In [ ]:
;pwd

In [ ]:
]activate ..

In [ ]:
using Revise

In [ ]:
using Apophis

In [ ]:
# using TaylorIntegration, LinearAlgebra # Apophis.jl @reexports TaylorIntegration and LinearAlgebra
using Plots, DelimitedFiles, Dates
using Statistics: mean, std
using AstroTime
using Printf

In [ ]:
using JLD
#recover integration from .jld file
vars = ["tv_jpl_integ", "xv1", "tvS1", "xvS1", "gvS1"] #names of variables
filename = string("Apophis_jt.jld")
for i in eachindex(vars)
    ex = Symbol(vars[i])
    @eval $ex = load($filename, vars[$i])
end

In [ ]:
apophis_radar_data_2005_2013 = process_radar_data_jpl("../Apophis_JPL_data.dat");

In [ ]:
apophis_data = apophis_radar_data_2005_2013[6:end] #take only the 2012-2013 subset

In [ ]:
#monostatic mode: check that each receiver and transmitter are the same
@show all( ismonostatic.(apophis_data) )

In [ ]:
#then, get the station codes
station_codes = [x.rcvr for x in apophis_data];

In [ ]:
#extract transmitter frequencies (MHz)
transmitter_freq_MHz = [x.freq for x in apophis_data];

In [ ]:
# get UTC receive times
tv_jpl_utc_jul = [datetime2julian(x.utcepoch) for x in apophis_data];

Let $\vec R$ denote the geocentric position of the observing station, $\vec \rho$ the topocentric position of the asteroid relative to the same station,  $\vec r_\mathrm{a}$ the geocentric position of the asteroid and $\vec r_\mathrm{E}$ the geocentric position of the asteroid. Then we have

$$
\vec r_\mathrm{a} =\vec r_\mathrm{E} + \vec R + \vec \rho
$$

Thus, the instantaneous topocentric range $\rho = |\vec \rho|$ of the asteroid may be computed as

$$
\rho = |\vec r_\mathrm{a} - \vec r_\mathrm{E} - \vec R| = \sqrt{(x_\mathrm{a} - x_\mathrm{E} - X)^2+(y_\mathrm{a} - y_\mathrm{E} - Y)^2+(z_\mathrm{a} - z_\mathrm{E} - Z)^2}
$$

And the instantaneous range rate $\dot \rho$ may be computed as:

$$
\dot \rho = \frac{1}{\rho}\vec{\rho} \cdot \dot{\vec{\rho}}
$$

# Delay/Doppler observations reduction

In [ ]:
niters = 3 # number of iterations in time delay recursive process
rad_ind = 2

In [ ]:
station_codes[rad_ind]

In [ ]:
UTCEpoch.(TDBEpoch.(tv_jpl_integ, origin=:julian))

In [ ]:
apophis_data[rad_ind].utcepoch

In [ ]:
tdelay, dshift = delay_doppler(
    xv1[rad_ind+1, :],
    apophis_data[rad_ind].rcvr,
    apophis_data[rad_ind].utcepoch,
    apophis_data[rad_ind].freq,
    niters
)

In [ ]:
tdelay_jpl, dshift_jpl = delay_doppler_jpleph(
    apophis_data[rad_ind].rcvr,
    apophis_data[rad_ind].utcepoch,
    apophis_data[rad_ind].freq,
    niters
)

In [ ]:
apophis_data[rad_ind].delay #DELAY observed value

In [ ]:
tdelay_jpl #DELAY computed value, JPL

In [ ]:
apophis_data[rad_ind].delay - tdelay_jpl #DELAY O-C, JPL

In [ ]:
tdelay() #DELAY computed value, TaylorIntegration.jl, with A2=0

In [ ]:
apophis_data[rad_ind].delay - tdelay() #DELAY O-C, TaylorIntegration, with A2=0

In [ ]:
apophis_data[rad_ind].delay - tdelay(-5.6844) #DELAY O-C, TaylorIntegration, with A2=-5.7

In [ ]:
apophis_data[rad_ind].doppler #DOPPLER observed value

In [ ]:
dshift_jpl #DOPPLER computed value, JPL

In [ ]:
apophis_data[rad_ind].doppler - dshift_jpl #DOPPLER O-C, JPL

In [ ]:
dshift() #DOPPLER computed value, TaylorIntegration.jl, with A2=0

In [ ]:
apophis_data[rad_ind].doppler - dshift() #DELAY O-C, TaylorIntegration, with A2=0

In [ ]:
apophis_data[rad_ind].doppler - dshift(-5.6844) #DELAY O-C, TaylorIntegration, with A2=-5.7

In [ ]:
#get time-delay, Doppler-shift values from integration

tdelay_v1 = Array{Taylor1{Float64}}(undef, length(apophis_data))
dshift_v1 = Array{Taylor1{Float64}}(undef, length(apophis_data))

for i in eachindex(apophis_data)
    tdelay_v1[i], dshift_v1[i] = delay_doppler(
        xv1[i+1, :],
        apophis_data[i].rcvr,
        apophis_data[i].utcepoch,
        apophis_data[i].freq,
        niters
    )
end

__tdelay_v1 = tdelay_v1
__dshift_v1 = dshift_v1

In [ ]:
# compute time-delay, Doppler-shift values from JPL ephemerides

tdelay_JPL_v1 = Array{Float64}(undef, length(apophis_data))
dshift_JPL_v1 = Array{Float64}(undef, length(apophis_data))

for i in eachindex(apophis_data)
    tdelay_JPL_v1[i], dshift_JPL_v1[i] = delay_doppler_jpleph(
        apophis_data[i].rcvr,
        apophis_data[i].utcepoch,
        apophis_data[i].freq,
        niters
    )
end

# Time-delay / Doppler shift plots

In [ ]:
delay_index = findall(x->x.delay_units=="us", apophis_data)
doppler_index = findall(x->x.doppler_units=="Hz", apophis_data);

In [ ]:
tdelay_jpl_obs = [x.delay for x in apophis_data][delay_index]
dshift_jpl_obs = [x.doppler for x in apophis_data][doppler_index]

tdelay_jpl_obs_sigma = [x.delay_sigma for x in apophis_data][delay_index]
dshift_jpl_obs_sigma = [x.doppler_sigma for x in apophis_data][doppler_index];

In [ ]:
# Observed minus computed (O-C) residuals: JPL DE430+s199

# absolute
residual_JPL_td = tdelay_jpl_obs - tdelay_JPL_v1[delay_index]  # (usec)
residual_JPL_ds = dshift_jpl_obs - dshift_JPL_v1[doppler_index] # (Hz)

# relative
rel_res_JPL_td = residual_JPL_td ./ tdelay_jpl_obs # (usec)
rel_res_JPL_ds = residual_JPL_ds ./ dshift_jpl_obs # (Hz)
;

In [ ]:
residual_JPL_td

In [ ]:
residual_JPL_ds

In [ ]:
dshift_v1 == __dshift_v1, tdelay_v1 == __tdelay_v1

In [ ]:
#Observed minus computed (O-C) residuals: TaylorIntegration.jl

tdelay_v1 = __tdelay_v1
dshift_v1 = __dshift_v1

#tdelay_v1 = __tdelay_v1 - (__tdelay_v1() - tdelay_JPL_v1)
#dshift_v1 = __dshift_v1 - (__dshift_v1() - dshift_JPL_v1)

#absolute
residual_td = tdelay_jpl_obs - (tdelay_v1[delay_index])  # (usec)
residual_ds = dshift_jpl_obs - (dshift_v1[doppler_index]) # (Hz)

#residual_td = tdelay_jpl_obs[3:end] - (tdelay_v1[1:end-5][delay_index[1:end-2]])  # (usec)
#residual_ds = dshift_jpl_obs[6:end] - (dshift_v1[1:end-5][doppler_index[1:end-5]]) # (Hz)

#relative
rel_res_td = residual_td ./ tdelay_jpl_obs # (usec)
rel_res_ds = residual_ds ./ dshift_jpl_obs # (Hz)

#rel_res_td = residual_td ./ tdelay_jpl_obs[3:end] # (usec)
#rel_res_ds = residual_ds ./ dshift_jpl_obs[6:end] # (Hz)

;

## Plots

In [ ]:
A2_plot = 0.0
scatter(
    tv_jpl_utc_jul[delay_index].-t0, tdelay_v1[delay_index](A2_plot),
    label="predicted (TaylorIntegration, A2 = $A2_plot x 10^-14 au/d^2)",
    marker=:cross
)
scatter!(
    tv_jpl_utc_jul[delay_index].-t0,
    tdelay_JPL_v1[delay_index],
    label="predicted (JPL DE430+s199)",
    marker=:xcross
)
scatter!(
    tv_jpl_utc_jul[delay_index].-t0,
    tdelay_jpl_obs,
    label="observed (Goldstone, Arecibo 2012-2013)",
    legend=:topleft,
    marker=:circle,
    markersize=3,
    yerror=dshift_jpl_obs_sigma
)
title!("Time delay vs time")
ylabel!("Total time delay [useconds]")
xlims!(1540, 1645)
xticks!(1540:20:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlabel!("Time")

In [ ]:
A2_plot = 0.0
scatter(
    tv_jpl_utc_jul[doppler_index].-t0, dshift_v1[doppler_index](A2_plot),
    label="predicted (TaylorIntegration, A2 = $A2_plot x 10^-14 au/d^2)",
    marker=:cross
)
scatter!(
    tv_jpl_utc_jul[doppler_index].-t0,
    dshift_JPL_v1[doppler_index],
    label="predicted (JPL DE430+s199)",
    marker=:xcross
)
scatter!(
    tv_jpl_utc_jul[doppler_index].-t0,
    dshift_jpl_obs,
    label="observed (Goldstone, Arecibo 2012-2013)",
    legend=:topright,
    marker=:circle,
    markersize=3,
    yerror=dshift_jpl_obs_sigma
)
title!("Doppler shift vs time")
ylabel!("Total Doppler shift [Hz]")
xlims!(1540, 1645)
xticks!(1540:20:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlabel!("Time")

In [ ]:
A2_plot = 0.0 #A2_lsqfit
scatter(
    tv_jpl_utc_jul[delay_index].-t0,
    residual_td(A2_plot), yerror = tdelay_jpl_obs_sigma,
    marker=:cross,
    label = "TaylorIntegration (A2 = $A2_plot x 10^-14 au/d^2)",
    legend = :topright
)
scatter!(
    tv_jpl_utc_jul[delay_index].-t0,
    residual_JPL_td, yerror = tdelay_jpl_obs_sigma,
    marker=:xcross,
    label = "JPL DE430+s199",
    legend = :topright
)
title!("Time delay residuals (O-C)")
ylabel!("Total time delay resid. (O-C) [us]")
xlims!(1540, 1645)
#ylims!(-215,210)
xticks!(1540:20:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlabel!("Time")
#savefig("delay_residuals_A2_minus_6.04.pdf")
#savefig("delay_residuals_A2_minus_5.7.pdf")

In [ ]:
A2_plot = 0.0
scatter(
    tv_jpl_utc_jul[doppler_index].-t0,
    residual_ds(A2_plot), yerror = dshift_jpl_obs_sigma,
    marker=:cross,
    label = "TaylorIntegration (A2 = $A2_plot x 10^-14 au/d^2)",
    leg=false
)
scatter!(
    tv_jpl_utc_jul[doppler_index].-t0,
    residual_JPL_ds, yerror = dshift_jpl_obs_sigma,
    marker=:xcross,
    label = "JPL DE430+s199",
    legend=:topleft
)
title!("Doppler shift residuals (O-C)")
ylabel!("Total Doppler shift resid. (O-C) [Hz]")
xlims!(1540, 1645)
xticks!(1540:20:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlabel!("Time")
#savefig("doppler_residuals_A2_minus_6.04.pdf")
#savefig("doppler_residuals_A2_minus_5.7.pdf")

In [ ]:
scatter(
    #tv_jpl_utc_jul[delay_index][1:end-3].-t0, residual_JPL_td[1:end-3], yerror = tdelay_jpl_obs_sigma[1:end-3],
    #tv_jpl_utc_jul[delay_index][end-3:end].-t0, residual_JPL_td[end-3:end], yerror = tdelay_jpl_obs_sigma[end-3:end],
    tv_jpl_utc_jul[delay_index].-t0, residual_JPL_td, yerror = tdelay_jpl_obs_sigma,
    marker=:xcross,
    label = "JPL DE430+s199",
    legend = :topright
)
ylabel!("Total time delay resid. (O-C) [us]")
#xlims!(1540, 1645)
#xticks!(1540:20:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlabel!("Time")
title!("Time delay residuals (O-C)")

In [ ]:
scatter(
    tv_jpl_utc_jul[doppler_index].-t0,
    residual_JPL_ds, yerror = dshift_jpl_obs_sigma,
    marker=:xcross,
    label = "JPL DE430+s199",
    legend=:bottomright
)
title!("Doppler shift residuals (O-C)")
ylabel!("Total Doppler shift resid. (O-C) [Hz]")
xlims!(1540, 1645)
xticks!(1540:20:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlabel!("Time")

In [ ]:
scatter(
    tv_jpl_utc_jul[delay_index].-t0,
    rel_res_JPL_td,
    yerror = tdelay_jpl_obs_sigma./tdelay_jpl_obs,
    label="JPL DE430+s199",
    marker=:xcross,
    legend=:topright
)
title!("Time delay relative resid. (O-C)/O")
xlims!(1540, 1645)
xticks!(1540:20:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlabel!("Time")

In [ ]:
scatter(
    tv_jpl_utc_jul[doppler_index].-t0,
    rel_res_JPL_ds,
    yerror = dshift_jpl_obs_sigma./dshift_jpl_obs,
    label="JPL DE430+s199",
    legend=:bottomright,
    marker=:xcross
)
title!("Doppler shift relative resid. (O-C)/O")
xlims!(1540, 1645)
#ylims!(-1e-7, 1e-7)
xticks!(1540:20:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlabel!("Time")

In [ ]:
A2_plot = 0.0
scatter(
    tv_jpl_utc_jul[delay_index].-t0,
    rel_res_td(A2_plot),
    yerror = tdelay_jpl_obs_sigma./tdelay_jpl_obs,
    label="TaylorIntegration (A2 = $A2_plot x 10^-14 au/d^2)",
    marker=:cross,
    legend=:topright
)
scatter!(
    tv_jpl_utc_jul[delay_index].-t0,
    rel_res_JPL_td,
    yerror = tdelay_jpl_obs_sigma./tdelay_jpl_obs,
    label="JPL DE430+s199",
    marker=:xcross,
    legend=:topright
)
title!("Time delay relative resid. (O-C)/O")
xlims!(1540, 1645)
xticks!(1540:20:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlabel!("Time")

In [ ]:
A2_plot = 0.0
scatter(
    tv_jpl_utc_jul[doppler_index].-t0,
    rel_res_ds(A2_plot),
    yerror = dshift_jpl_obs_sigma./dshift_jpl_obs,
    label="TaylorIntegration (A2 = $A2_plot x 10^-14 au/d^2)",
    legend=:bottomright,
    marker=:cross
)
scatter!(
    tv_jpl_utc_jul[doppler_index].-t0,
    rel_res_JPL_ds,
    yerror = dshift_jpl_obs_sigma./dshift_jpl_obs,
    label="JPL DE430+s199",
    legend=:bottomright,
    marker=:xcross
)
title!("Doppler shift relative resid. (O-C)/O")
xlims!(1540, 1645)
xticks!(1540:20:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlabel!("Time")

#check that HORIZONS (s#199) and downloaded SPK file for Apophis are the same
#according to this check, they differ at the end of the evaluated times by ~ 2m in position
tv_spkvhor = range(2454733.5, length=1001, stop=2456367.5)
apophis_spk_arr = Array{Float64}(undef, length(tv_spkvhor), 6)
for i in eachindex(tv_spkvhor)
    apophis_spk_arr[i,:] .= Apophis.apophis_pv(tv_spkvhor[i])
end
apophis_horizons_arr = readdlm("../wld103954.15")
@show norm((apophis_spk_arr .- apophis_horizons_arr)[end,1:3])
@show norm((apophis_spk_arr .- apophis_horizons_arr)[end,4:6])
;

# Yarkovsky $A_2$ coefficient estimation

Here, we will compute, for a single time delay observation, the value of $A_2$ which makes the O-C residual of that observation equal to zero:

In [ ]:
#observed value
tdelay_jpl_obs[1]

In [ ]:
# computed value (full solution)
tdelay_v1[delay_index[1]] # τ_c(A2) = p(A2)

In [ ]:
# residual (A2 = 0)
tdelay_jpl_obs[1] - tdelay_v1[delay_index[1]]()

In [ ]:
# substract A2=0 solution from τ_c(A2) (i.e., compute τ_c(A2)-τ_c(A2=0) )
p_τ_1 = tdelay_v1[delay_index[1]]-tdelay_v1[delay_index[1]]()

In [ ]:
# Let p_τ_1(A2) be the above polynomial expression, then we invert it so we get A2(δτ):
A2_δτ_1 = inverse(p_τ_1)

In [ ]:
#Now, we evaluate the polynomial at the corresponding residual at the A2=0 TaylorIntegration solution:
A2_δτ_1(residual_td[1]())

Now, we follow the same strategy for all the 2012-2013 radar astrometry for Apohpis (delay and Doppler):

In [ ]:
td_us = tdelay_v1[delay_index]
ds_Hz = dshift_v1[doppler_index];

In [ ]:
A2_δτ_v = (  inverse.( td_us-td_us() )  ) # vector of A2(δτ) polynomials at each delay observation;
A2_δν_v = (  inverse.( ds_Hz-ds_Hz() )  ) # vector of A2(δν) polynomials at each Doppler observation;

In [ ]:
A2_del_v = map((x,y)->x(y), A2_δτ_v, residual_td()); # A2(δτ) polynomials evaluated at the O-C time delay residuals;
A2_dop_v = map((x,y)->x(y), A2_δν_v, residual_ds()); # A2(δν) polynomials evaluated at the O-C Doppler shift residuals;

In [ ]:
A2_del_v

In [ ]:
A2_dop_v

In [ ]:
mean(A2_del_v), std(A2_del_v)

In [ ]:
mean(A2_dop_v), std(A2_dop_v)

In [ ]:
A2_all = vcat(A2_del_v, A2_dop_v)

In [ ]:
A2_mean_all = mean(A2_all)
A2_std_all = std(A2_all)
A2_mean_all, A2_std_all

In [ ]:
scatter(
    tv_jpl_utc_jul[delay_index][1:end-3].-t0,
    A2_del_v[1:end-3],
    marker=:cross,
    label="A2 est. (Goldstone delay obs.)"
)
scatter!(
    tv_jpl_utc_jul[delay_index][end-2:end].-t0,
    A2_del_v[end-2:end],
    marker=:cross,
    label="A2 est. (Arecibo delay obs.)"
)
ylabel!("A2(dt) x 10^14 [au/d^2]")
xlims!(1540, 1645)
xticks!(1540:20:1645, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlabel!("Time")

In [ ]:
scatter(
    tv_jpl_utc_jul[doppler_index][1:end-7].-t0,
    A2_dop_v[1:end-7],
    marker=:cross,
    label="A2 est. (Goldstone Doppler obs.)"
)
scatter!(
    tv_jpl_utc_jul[doppler_index][end-6:end].-t0,
    A2_dop_v[end-6:end],
    marker=:cross,
    label="A2 est. (Arecibo Doppler obs.)",
    leg=:bottomleft
)
ylabel!("A2(df) x 10^14 [au/d^2]")
xlims!(1540, 1645)
xticks!(1540:20:1645, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlabel!("Time")

In [ ]:
scatter(
    tv_jpl_utc_jul[delay_index][1:end-3].-t0,
    A2_del_v[1:end-3],
    marker=:cross,
    label="A2 est. (Goldstone delay obs.)"
)
scatter!(
    tv_jpl_utc_jul[doppler_index][1:end-7].-t0,
    A2_dop_v[1:end-7],
    marker=:cross,
    label="A2 est. (Goldstone Doppler obs.)"
)
scatter!(
    tv_jpl_utc_jul[delay_index][end-2:end].-t0,
    A2_del_v[end-2:end],
    marker=:xcross,
    label="A2 est. (Arecibo delay obs.)"
)
scatter!(
    tv_jpl_utc_jul[doppler_index][end-6:end].-t0,
    A2_dop_v[end-6:end],
    marker=:xcross,
    label="A2 est. (Arecibo Doppler obs.)",
    leg=:bottomleft
)
ylabel!("A2 x 10^14 [au/d^2]")
xlims!(1540, 1645)
ylims!(-105,5)
xticks!(1540:20:1645, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlabel!("Time")

In [ ]:
A2_NO_Arecibo_dop = vcat(A2_del_v, A2_dop_v[1:end-7]);

In [ ]:
A2_mean_NO_Arecibo_dop = mean(A2_NO_Arecibo_dop)
A2_std_NO_Arecibo_dop = std(A2_NO_Arecibo_dop)
A2_mean_NO_Arecibo_dop, A2_std_NO_Arecibo_dop

# Intervals

In [ ]:
using IntervalArithmetic

In [ ]:
residual_td_interval = interval.(
    (tdelay_jpl_obs-tdelay_jpl_obs_sigma)-tdelay_v1[delay_index](), 
    (tdelay_jpl_obs+tdelay_jpl_obs_sigma)-tdelay_v1[delay_index]()
);

In [ ]:
residual_ds_interval = interval.(
    (dshift_jpl_obs-dshift_jpl_obs_sigma)-dshift_v1[doppler_index](),
    (dshift_jpl_obs+dshift_jpl_obs_sigma)-dshift_v1[doppler_index]()
);

In [ ]:
A2_del_v_interval = map((x,y)->x(y), A2_δτ_v, residual_td_interval)

In [ ]:
A2_dop_v_interval = map((x,y)->x(y), A2_δν_v, residual_ds_interval)

In [ ]:
scatter(
    tv_jpl_utc_jul[delay_index][1:end-3].-t0,
    mid.(A2_del_v_interval[1:end-3]),
    yerror=radius.(A2_del_v_interval[1:end-3]),
    label="delay (Goldstone)",
    leg=:topright,
    marker=:cross
)
scatter!(
    tv_jpl_utc_jul[delay_index][end-2:end].-t0,
    mid.(A2_del_v_interval[end-2:end]),
    yerror=radius.(A2_del_v_interval[end-2:end]),
    label="delay (Arecibo)",
    leg=:topright,
    marker=:cross
)
#ylims!(minimum(A2_del_v)-1,0)
ylabel!("A2(dt) x 10^14 [au/d^2]")
xlims!(1540, 1645)
xticks!(1540:20:1645, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlabel!("Time")
title!("A2 estimation from polynomial inversion (delay data)")
#savefig("A2_poly_inv_delay.pdf")

In [ ]:
scatter(
    tv_jpl_utc_jul[doppler_index][1:end-7].-t0,
    mid.(A2_dop_v_interval[1:end-7]),
    yerror=radius.(A2_dop_v_interval[1:end-7]),
    label="Doppler (Goldstone)",
    marker=:cross
)
scatter!(
    tv_jpl_utc_jul[doppler_index][end-6:end].-t0,
    mid.(A2_dop_v_interval[end-6:end]),
    yerror=radius.(A2_dop_v_interval[end-6:end]),
    label="Doppler (Arecibo)",
    marker=:cross,
    leg=:bottomleft
)
ylabel!("A2(df) x 10^14 [au/d^2]")
xlims!(1540, 1645)
xticks!(1540:20:1645, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlabel!("Time")

In [ ]:
A2_v_NO_Arecibo_dop_interval = vcat(A2_del_v_interval, A2_dop_v_interval[1:end-7])

In [ ]:
A2_mean_NO_Arecibo_dop_interval = mean(A2_v_NO_Arecibo_dop_interval)
A2_std_NO_Arecibo_dop_interval = std(A2_v_NO_Arecibo_dop_interval)
A2_mean_NO_Arecibo_dop_interval, A2_std_NO_Arecibo_dop_interval

In [ ]:
scatter(
    tv_jpl_utc_jul[delay_index][1:end-3].-t0,
    mid.(A2_del_v_interval[1:end-3]),
    yerror=radius.(A2_del_v_interval[1:end-3]),
    marker=:cross,
    leg=:bottomleft,
    label="delay (Goldstone)"
)
scatter!(
    tv_jpl_utc_jul[doppler_index][1:end-7].-t0,
    mid.(A2_dop_v_interval[1:end-7]),
    yerror=radius.(A2_dop_v_interval[1:end-7]),
    marker=:cross,
    label="Doppler (Goldstone)"
)
scatter!(
    tv_jpl_utc_jul[delay_index][end-2:end].-t0,
    mid.(A2_del_v_interval[end-2:end]),
    yerror=radius.(A2_del_v_interval[end-2:end]),
    marker=:xcross,
    leg=:bottomleft,
    label="delay (Arecibo)"
)
scatter!(
    tv_jpl_utc_jul[doppler_index][end-6:end].-t0,
    mid.(A2_dop_v_interval[end-6:end]),
    yerror=radius.(A2_dop_v_interval[end-6:end]),
    marker=:xcross,
    label="Doppler (Arecibo)"
)
A2_label_str = @sprintf "A2 (fit): %.5f  x 10^-14 au/d^2" A2_lsqfit_deldop
plot!([1540, 1640], [A2_lsqfit_deldop, A2_lsqfit_deldop], label=A2_label_str)
ylabel!("A2 x 10^14 [au/day^2]")
xticks!(1540:20:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlabel!("Time")
title!("Brozovic et al. (2018) data (Goldstone and Arecibo)")
#savefig("A2_fit_radar_all.pdf")

In [ ]:
scatter(
    tv_jpl_utc_jul[delay_index][1:end-3].-t0,
    mid.(A2_del_v_interval[1:end-3]),
    yerror=radius.(A2_del_v_interval[1:end-3]),
    marker=:cross,
    leg=:bottomleft,
    label="delay (Goldstone)"
)
scatter!(
    tv_jpl_utc_jul[doppler_index][1:end-7].-t0,
    mid.(A2_dop_v_interval[1:end-7]),
    yerror=radius.(A2_dop_v_interval[1:end-7]),
    marker=:cross,
    label="Doppler (Goldstone)"
)
scatter!(
    tv_jpl_utc_jul[delay_index][end-2:end].-t0,
    mid.(A2_del_v_interval[end-2:end]),
    yerror=radius.(A2_del_v_interval[end-2:end]),
    marker=:xcross,
    leg=:bottomright,
    label="delay (Arecibo)"
)
scatter!(
    tv_jpl_utc_jul[doppler_index][end-6:end].-t0,
    mid.(A2_dop_v_interval[end-6:end]),
    yerror=radius.(A2_dop_v_interval[end-6:end]),
    marker=:xcross,
    label=""#"Doppler (Arecibo)"
)
A2_label_str = @sprintf "A2 (fit): %.5f  x 10^-14 au/d^2" A2_lsqfit_deldop_NO_Arecibo_dop
#plot!([1540, 1645], [A2_lsqfit_deldop, A2_lsqfit_deldop], label="A2 (fit): $A2_lsqfit_deldop x 10^-14 au/d^2")
plot!([1540, 1645], [A2_lsqfit_deldop, A2_lsqfit_deldop], label=A2_label_str)
ylims!(-20,0)
ylabel!("A2 x 10^14 [au/day^2]")
xlims!(1540, 1645)
xlabel!("Time")
xticks!(1540:20:1645, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1645)))))
title!("A2 estimation from radar astrometry")
#title!("Brozovic et al. (2018) data (no Arecibo Doppler astrom. )")
#savefig("A2_fit_no_Arecibo_doppler.pdf")

## Least squares

In [ ]:
res_sq_del = ( tdelay_jpl_obs .- tdelay_v1[delay_index] ).^2
res_sq_dop = ( dshift_jpl_obs .- dshift_v1[doppler_index] ).^2
res_sq_deldop = vcat(res_sq_del, res_sq_dop)
res_sq_deldop_NO_Arecibo_dop = vcat(res_sq_del, res_sq_dop[1:end-7]); #remove Arecibo Doppler "outliers"

In [ ]:
Q_A2_del = sum(res_sq_del)/length(res_sq_del)

In [ ]:
Q_A2_dop = sum(res_sq_dop)/length(res_sq_dop)

In [ ]:
Q_A2_deldop = sum(res_sq_deldop)/length(res_sq_deldop)

In [ ]:
Q_A2_deldop_NO_Arecibo_dop = sum(res_sq_deldop_NO_Arecibo_dop)/length(res_sq_deldop_NO_Arecibo_dop)

In [ ]:
function taylor_roots(pol::Taylor1{T}, x0::T; niters::Int=10) where {T<:Real}
    dpol = differentiate(pol)
    dpol2 = differentiate(dpol)
    xnewton::T = x0
    @show xnewton
    for i in 1:niters
        xnewton -= dpol(xnewton)/dpol2(xnewton) 
        @show xnewton, dpol(xnewton)
    end
    return xnewton
end

In [ ]:
A2_lsqfit_del = taylor_roots(Q_A2_del, 0.0, niters=5)

In [ ]:
A2_lsqfit_dop = taylor_roots(Q_A2_dop, 0.0, niters=5)

In [ ]:
A2_lsqfit_deldop = taylor_roots(Q_A2_deldop, 0.0, niters=5)

In [ ]:
A2_lsqfit_deldop_NO_Arecibo_dop = taylor_roots(p_A2_deldop_NO_Arecibo_dop, 0.0, niters=5)

In [ ]:
#A2_del_vals = collect(-10:0.01:10);
#A2_vals = range(-5.68444, length=100, stop=-5.68442)
#A2_vals = range(-7.5, length=200, stop=-5)
#A2_vals = range(-5.7, length=200, stop=-5.65)
A2_vals = range(-5.9, length=2000, stop=-5.5)

In [ ]:
plot(A2_vals, Q_A2_deldop.(A2_vals), leg=false)
xlabel!("A2 x 10^14 [au/day^2]")
ylabel!("Q(A2) [units?]")
title!("LS cost function vs A2")
ylims!(-1, 11)
xlims!(-5.9, -5.5)
#plot(A2_vals, p_A2_dop.(A2_vals))
#plot!(A2_vals, p_A2_deldop.(A2_vals))
#xlims!(-7.5, -5.0)